# Watson Machine Learning - Style Transfer

In this notebook we'll walk through connecting to the [Watson Machine Learning service](https://www.ibm.com/cloud/machine-learning) and executing a **Style Transfer** workload on the IBM Cloud **GPUs** using **PyTorch**

The workflow is as follows: 

1. Install WGET
2. Get the Pytorch script for Style Transfer
3. Load our images
4. Add credentials for writing to the IBM Cloud Object Storage bucket
5. Connect to the Watson Machine Learning (WML) runtime to execute our job on the GPU
6. Set up the WML job
7. Execute the job
8. Admire your results

**PROTIP** -- *In this example we're taking the style from one image and trying to apply it to the content of another. This works best if you pick a really extreme style image. Bright primary colours are great, famous works of art are fun too. For your content image we'd suggest something with people/objects in. That way you'll end up with a distinctive result.* 


## Installing WGET

The first step of the process is to install `wget` in vritual machine where your notebook is running. This will allow us to get the PyTorch code from GitHub we'll be using to perform the Style Transfer on our images.

To install wget - run the code cell below:

In [1]:
!pip install --upgrade wget

Requirement already up-to-date: wget in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages


## Download the PyTorch Code

Now we've got WGET we need to head out to GitHub and grab the PyTorch script we'll be using. If you want to see the script in more detail, you can see it [here](https://github.com/ChrisParsonsDev/wml-pytorch-style-transfer/blob/master/pytorch-model/style-transfer.py).

Often when we're building Machine Learning or Deep Learning models, we'll right the code in a notebook like this. Notbeooks are a great tool for visualising the data science workflow, you can see exactly what's going on, and how your data changes as you work.

In this instance however, we're just interested in submitting a job to a GPU, to speed up execution time. You'd probably follow a similar workflow to this if you were trying to scale an app to production (where you're no longer trying to visualise everything) or if you wanted your code separated from your data science workflow. Data in one place, code in another. 

The script does a few things, here's a step by step of what's going on behind the scenes: 

**1. Command Line Arguments**

As you'll see later in this notebook, the PyTorch script can handle a number of command line arguments. These let us specify our own style and content images, how many iterations through the photos we conduct as "training" and how often we want to see output. 

**2. Load Image Files**

The next thing we need to do is load in your images. For style transfer we need two images! One, which has the "content" or the subject we're interested in and another with the "style" or pattern we'd like to copy across. The code also takes care of resizing your images so they're the same size (and you won't get any strange results). They're also made to be `400px` along their longest edge. The larger the image, the longer the training will take. 

**3. Download our Neural Network**

In this example we're using [VGG](http://www.robots.ox.ac.uk/~vgg/), an existing neural network, to do the heavy lifting for our training. To make things simple VGG is one of the "sample" models included in PyTorch - we can simply load in a pretrained version and use it as we wish.

**4. Select the "Content" and "Style" layers**

We're pulling out a few layers of the network here. One to represent the subject of the image we want (that's our content) and 5 more to represent the "style" of a different image we'd like to apply to that content. 

**5. Perform Style Transfer**

This is the real meat of the code - we're going to iterate a few 1000 times through both our style and content images and try to apply the "Style" to the "Content".

**6. Save The Output**

Finally, during the training process, the script will checkpoint and save the "transferred" images to your CoS bucket. This will let you see how things change/progress through the various epochs. It's really cool to watch how the algorithm learns, and watch your content become way more stylised. 


So - let's head out to GitHub and download the `.zip` file with the PyTorch script in it. To get the code - run the cell below:

In [2]:
import os
import wget

#Import files 
filename = 'pytorch-model.zip'
url = 'https://github.com/ChrisParsonsDev/wml-pytorch-style-transfer/blob/master/pytorch-model.zip?raw=true'

if not os.path.isfile( filename ): wget.download( url )

print('Files downloaded!')

Files downloaded!


## Read Credentials

Ok, so we've got the code, now we need our images. 

Luckily Watson Studio makes getting the "read" credentials for our photos into our notebook **really simple**. You'll only need to do this once (as the credentials are the same for both images). 

This will allow our notebook to pass the style and content photos on to Watson Machine Learning and the GPUs for the style transfer itself.

Watch out though, you'll need to make sure you rename the dictionary `image_credentials` (as with our example below). If you leave it as `credentials_1` (default). Stuff wont work and you'll get a whole bunch of errors. 

Follow the tutorial on [GitHub](https://github.com/ChrisParsonsDev/wml-pytorch-style-transfer) and insert your credentials into the code cell below. 

It should look a little like this: 

````
image_credentials = {
    'IBM_API_KEY_ID': '***',
    'IAM_SERVICE_ID': '***',
    'ENDPOINT': '***',
    'IBM_AUTH_ENDPOINT': '***',
    'BUCKET': '***',
    'FILE': '***'
}

````

Once you've done that. Run the block and move on down..

In [3]:
# Insert your image credentials here and name them image_credentials



## Write Credentials

Where's the fun in machine learning if you can't write your output anywhere? 

In order to grant the Watson Machine Learning API permission to write back out to our CoS bucket we need to grab the credentials and insert them into the code block below. 

The [tutorial](https://github.com/ChrisParsonsDev/wml-pytorch-style-transfer) tells you how you can get your specific credentials. 

Make sure you call them `cos_credentials` as we did, just to keep everything working properly. Of course, you're more than welcome to go all the way through and rename the variable if you want. It just seems like a whole lot more work to us. ;) 

Once you've replaced the placeholder below with your details, run the code cell below: 

In [4]:
# Insert your Cloud Object Storage credentials here!

cos_credentials = {
  "apikey": '***',
  "cos_hmac_keys": {
    "access_key_id": '***',
    "secret_access_key": '***'
  },
  "endpoints": '***',
  "iam_apikey_description": '***',
  "iam_apikey_name": '***',
  "iam_role_crn": '***',
  "iam_serviceid_crn": '***',
  "resource_instance_id": '***'
}

## Watson Machine Learning

Here's where the fun **really** starts. Watson Machine Learning. 

Watson Machine Learning is a really powerful toolkit that gives us easy access to GPU compute resource for Machine Learning tasks in the cloud. It makes speeding up your machine learning trivial. 

In order to start submitting jobs to the API we need to go ahead and get the credentials though, this stops just anyone using your GPU resources and building all the AI. 

The [tutorial](https://github.com/ChrisParsonsDev/wml-pytorch-style-transfer) tells you how you can get your specific credentials. 

Make sure you call them `wml_credentials` as we did, just to keep everything working properly. 

Finally this code cell will configure an instance of the API called `client` that we'll set up for our PyTorch style transfer workload. 

Once you've replaced the placeholder below with your details, run the code cell below: 

In [5]:
# Insert your Watson Machine Learning credentials here!

from watson_machine_learning_client import WatsonMachineLearningAPIClient

wml_credentials = {
  "apikey": '***',
  "iam_apikey_description": '***',
  "iam_apikey_name": '***',
  "iam_role_crn": '***',
  "iam_serviceid_crn": '***',
  "instance_id": '***',
  "password": '***',
  "url": '***',
  "username": '***'
}

client = WatsonMachineLearningAPIClient( wml_credentials )

print('Watson Machine Learning credentials loaded')


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Watson Machine Learning credentials loaded


2019-01-10 10:47:33,593 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.
2019-01-10 10:48:07,466 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.


## Custom Settings

Alright, so we're nearly there, we've got access to our images, we can write to the CoS bucket and Watson Machine Learning is set up and ready to go. 

There are just a few parameters we need to tweak to make sure everything works seemlessly. We've included 3 variables in the code cell below to make this as easy as possible. 

**style_image_name**

For style transfer to work properly we need to supply the system with two images. Firstly, our "style" image. This photo contains the style and patterns we want to apply to our content image. Make sure the string here **EXACTLY MATCHES** the filename for the image you uploaded earlier. 

Oh, and check the file exension too, otherwise we'll end up with a file not found error, which would be really embarrassing. 

**content_image_name**

Now we've got our style image sorted it's time for the content. This is the "subject" we want to apply the style to. Update the variable below so the image name matches the image you uploaded to your CoS bucket earlier on. 

Make sure you check that file exension.. 

**training_iterations**

This is more commonly referred to as `epochs`. Really it's the number of times the code will run through your images attempting to improve the style transfer. The higher this number, the more style we can expect to be applied to our content image. Of course, the more iterations we have, the longer it will take. 

We've used 2000 as a default, but feel free to experiment with this number and see what images you can create. 

Update the parameters in the code cell below to match your images. 

In [7]:
style_image_name = "style.png"
content_image_name = "content.jpeg"
training_iterations = "2000"

## Configure the Watson Machine Learning API

That's all the setup complete, we simply need to configure and run our job. If you execute the code cell below we'll set up the Watson Machine Learning service to run our PyTorch code. 

You can see the different parameters here are configuring differeng parts of the API. 

**FRAMEWORK_NAME**

This is the Machine Learning framework we'd like to use. Watson Machine Learning supports the majority of the Open Source ml/dl frameworks like TensorFlow and PyTorch. 

**FRAMEWORK_VERSION**

Do you want a specific version of the framework, or are you happy with the latest and greatest? You can specify that here. 

**RUNTIME_NAME**

Python or R? That's not a debate we want to get in to, but it's all fully customisable through the API. 

**RUNTIME_VERSION**

You can specify a specific version of the runtime to use, if you've got dependancies on a different version of Python for instance, you can manage that here. 

**EXECUTION_COMMAND**

This is the command you'd like the Watson Machine Learning API to execute. Here we're getting the PyTorch script, and specifying the input images and training iterations as command line arguments. 

Run the code cell below to configure your Watson Machine Learning job. 

In [6]:
metadata = {
    client.repository.DefinitionMetaNames.NAME              : "pytorch-styletransfer-wml",
    client.repository.DefinitionMetaNames.AUTHOR_EMAIL      : "****",
    client.repository.DefinitionMetaNames.FRAMEWORK_NAME    : "pytorch",
    client.repository.DefinitionMetaNames.FRAMEWORK_VERSION : "1.0",
    client.repository.DefinitionMetaNames.RUNTIME_NAME      : "python",
    client.repository.DefinitionMetaNames.RUNTIME_VERSION   : "3.5",
    client.repository.DefinitionMetaNames.EXECUTION_COMMAND : "python3 ./pytorch-model/style-transfer.py --styleImageFile ${{DATA_DIR}}/{} --contentImageFile ${{DATA_DIR}}/{} --trainingIters {}".format(style_image_name,content_image_name,training_iterations)
}

definition_details = client.repository.store_definition( "pytorch-model.zip", meta_props=metadata )
definition_uid     = client.repository.get_definition_uid( definition_details )

print( "Model definition: ", definition_uid )

Model definition:  13709e72-d72b-401c-b9c0-657b193aa2f8


## Watson Machine Learning Output

Now we've got our model set up, we need to tell the API where to save the output. We imported the credentials earlier in this notebook, so now we simply need to pass them over to the Watson Machine Learning API. 

Run this code cell below to import the storage credentials. 

In [9]:
metadata = {
client.training.ConfigurationMetaNames.NAME         : "pytorch-styletransfer-wml",
client.training.ConfigurationMetaNames.AUTHOR_EMAIL : "*****",
client.training.ConfigurationMetaNames.TRAINING_DATA_REFERENCE : {
   "connection" : { 
      "endpoint_url"      : image_credentials['ENDPOINT'],
      "access_key_id"     : cos_credentials["cos_hmac_keys"]["access_key_id"],
      "secret_access_key" : cos_credentials["cos_hmac_keys"]["secret_access_key"]
      },
   "source" : { 
      "bucket" : image_credentials['BUCKET'],
      },
      "type" : "s3"
   },
client.training.ConfigurationMetaNames.TRAINING_RESULTS_REFERENCE: {
   "connection" : {
      "endpoint_url"      : image_credentials['ENDPOINT'],
      "access_key_id"     : cos_credentials["cos_hmac_keys"]["access_key_id"],
      "secret_access_key" : cos_credentials["cos_hmac_keys"]["secret_access_key"]
      },
      "target" : {
         "bucket" : image_credentials['BUCKET'],
      },
      "type" : "s3"
   }
}

## Run Forrest Run

We're there! Everyting is configured, we've got our images loaded in and we're good to go. 

Run the code cell below to submit the style transfer job to the GPU. 

In [10]:
run_details = client.training.run( definition_uid, meta_props=metadata )
run_uid     = client.training.get_run_uid( run_details )
print( "run_uid: ", run_uid )

run_uid:  model-hqwass1f


## Status Update

To view the status of your job at any time, run the code cell below. 

Training, on the free tier, should take approximately 6 minutes (assuming you used the defaults).

You should see the status change from:

1. **Pending** - we're waiting for a GPU to be free to run our job and setting up the runtime.
2. **Running** - woohoo! We're transferring the style.
3. **Completed** - all done! 

When it's all done, check out the CoS bucket to see your results. As detailed [here](https://github.com/ChrisParsonsDev/wml-pytorch-style-transfer)

In [31]:
client.training.get_status( run_uid )

{'current_at': '2019-01-10T11:06:21Z',
 'message': 'training-8feH238mg: ',
 'metrics': [],
 'running_at': '2019-01-10T10:50:34Z',
 'state': 'running',
 'submitted_at': '2019-01-10T10:48:08Z'}